In [1]:
import numpy as np
import time
import os
import h5py
import glob
import IPython.display
import matplotlib.pyplot as plt
np.random.seed(1337)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Merge
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from PIL import Image
from keras.layers.normalization import BatchNormalization

Using Theano backend.
Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 95.0% of memory, cuDNN not available)


In [2]:
# Define the parameters for training
batch_size = 128
nb_classes = 2
nb_epoch = 20

# input image dimensions
img_rows, img_cols = 11, 11

# Volume of the training set
sample_number = 100000

# number of conv filters to use
nb_filters = 112

# CNN kernel size
kernel_size = (3,3)

1. Training set contains 50,000 pairs of positive-matching samples and 50,000 pairs of negative-matching samples. The volume of our training set is far smaller than the paper (i.e., they use ~260,000,000 samples).
2. We load all the training set into a 3D matrix.
3. Loading 50,000 pairs of images takes approximately 600 s.
4. Loading 100,000 pairs of images takes approximately 2282 s.

In [3]:
# load the patches
X1_train = np.zeros((sample_number, img_rows, img_cols))
X2_train = np.zeros((sample_number, img_rows, img_cols))
y_train = np.zeros((sample_number,))

tic = time.time()
# Load the training set.
hdf5TrainPatchesPath = "/home/qw2208/research/trainPatches.hdf5"
with h5py.File(hdf5TrainPatchesPath, "r") as f1:
    for i in xrange(sample_number/2):
        X1_train[2*i,:,:] = f1['left/'+str(i)][()]
        X1_train[(2*i+1),:,:] = f1['left/'+str(i)][()]
        X2_train[2*i,:,:] = f1['rightNeg/'+str(i)][()]
        X2_train[(2*i+1),:,:] = f1['rightPos/'+str(i)][()]
        y_train[2*i] = 0
        y_train[2*i+1] = 1
    
toc = time.time()
print "Time for loading the training set: ", toc-tic

Time for loading the training set:  89.9266209602


1. Here some additional preprocess methods like rotation etc. could be added.
2. Normalization missed.

In [4]:
# Resize the dataset (Trivial)
if K.image_dim_ordering() == 'th':
    X1_train = X1_train.reshape(X1_train.shape[0], 1, img_rows, img_cols)
    X2_train = X2_train.reshape(X2_train.shape[0], 1, img_rows, img_cols)
    input_shape = (1,img_rows, img_cols)
else:
    X1_train = X1_train.reshape(X1_train.shape[0], img_rows, img_cols, 1)
    X2_train = X2_train.reshape(X2_train.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols,1)

In [5]:
# Briefly check some patches.
# Positive-matching patches are expected to be of similar features.
# We store two left patches in X1_train.
# One for matching the positve right patch in X2_train. 
# The other for matching negative right patch in X2_train.

for i in xrange(20,30):
    print 'Check {}'.format(i-19)
    print X1_train[2*i][0]
    # print (X1_train[2*i+1][0]-X2_train[2*i+1][0])

X1_train = X1_train.astype('float32')
X2_train = X2_train.astype('float32')

Check 1
[[ 0.53156143  0.53156143  0.56323642  0.51572394  0.80079854  0.64242381
   0.27816182  0.24648687  0.24648687  0.18313695  0.24648687]
 [-0.32366228 -0.26031238 -0.07026266  0.40486163  1.22841036  1.13338554
   0.27816182  0.10394958  0.0881121   0.0247622   0.0881121 ]
 [ 0.75328612  0.75328612  0.75328612  0.80079854  0.78496104  0.80079854
   0.81663603  0.83247352  0.78496104  0.72161114  0.75328612]
 [ 0.76912361  0.72161114  0.80079854  0.78496104  0.78496104  0.84831101
   0.73744863  0.83247352  0.76912361  0.73744863  0.81663603]
 [ 0.73744863  0.68993622  0.70577371  0.73744863  0.68993622  0.67409873
   0.72161114  0.73744863  0.68993622  0.64242381  0.62658632]
 [ 0.151462    0.0881121   0.0881121   0.11978705  0.0247622   0.00892472
  -0.00691276 -0.00691276 -0.10193761 -0.13361257 -0.16528752]
 [-0.35533723 -0.33949977 -0.33949977 -0.40284967 -0.40284967 -0.38701218
  -0.40284967 -0.38701218 -0.38701218 -0.38701218 -0.40284967]
 [-0.40284967 -0.45036212 -0.4503

This neural network is working finely and ends up with a training accuracy of more than 90%. Some units are missing:
1. Data augment: Rotation, transformation and etc.
2. Volume of the training set. 
3. I haven't split the training set into training set and validation set. And the evaluation of the network is based on the whole training set, which is not permitted.

In [1]:
# https://keras-cn.readthedocs.io/en/latest/getting_started/sequential_model/#merge
left_branch = Sequential()
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same', input_shape=input_shape))
left_branch.add(Activation('relu'))
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
left_branch.add(Activation('relu'))
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
left_branch.add(Activation('relu'))
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
left_branch.add(Activation('relu'))

right_branch = Sequential()
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same', input_shape=input_shape))
right_branch.add(Activation('relu'))
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
right_branch.add(Activation('relu'))
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
right_branch.add(Activation('relu'))
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
right_branch.add(Activation('relu')) 

merged = Merge([left_branch, right_branch], mode='concat', concat_axis=1)
fc = Sequential()
fc.add(merged)
fc.add(Flatten())
fc.add(Dense(384, activation='relu'))
fc.add(Dense(384, activation='relu'))
fc.add(Dense(384, activation='relu'))

fc.add(Dense(1, activation='sigmoid'))

fc.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
fc.fit([X1_train,X2_train], y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=0, shuffle=True, callbacks=[TQDMNotebookCallback()])

# Evaluate the result based on the training set
score = fc.evaluate([X1_train,X2_train], y_train, verbose=0)
print('Test score: ', score[0])
print('Test accuracy: ', score[1])

NameError: name 'Sequential' is not defined

We need to store the trained network. This is not finished.

In [7]:
fc.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 112, 11, 11)   1120                                         
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 11, 11)   0                                            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 112, 11, 11)   113008                                       
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 112, 11, 11)   0                                            
___________________________________________________________________________________________

In [8]:
fc.save('/home/qw2208/research/weights.hdf5')